In [4]:
# This notebook serves as a testing space for visualiser functions
import numpy as np
##import scipy
#import pandas
import matplotlib as mpl
import matplotlib.pyplot as plt
import allel
print('scikit-allel', allel.__version__)

scikit-allel 1.3.11


In [5]:
#store VCF data as a variable:
from os import path
data_path = path.realpath("./Data/afr-small.vcf")
data_set = allel.read_vcf(data_path, fields = '*')

##### VCF Data Keys

In [6]:
#Print all fields and key-value-pairs that can access NumPy array data 
sorted(data_set.keys())

['calldata/GT',
 'samples',
 'variants/AA',
 'variants/AC',
 'variants/AF',
 'variants/AFR_AF',
 'variants/ALT',
 'variants/AMR_AF',
 'variants/AN',
 'variants/CHROM',
 'variants/CIEND',
 'variants/CIPOS',
 'variants/CS',
 'variants/DP',
 'variants/EAS_AF',
 'variants/END',
 'variants/EUR_AF',
 'variants/FILTER_PASS',
 'variants/ID',
 'variants/IMPRECISE',
 'variants/MC',
 'variants/MEINFO',
 'variants/MEND',
 'variants/MLEN',
 'variants/MSTART',
 'variants/NS',
 'variants/POS',
 'variants/QUAL',
 'variants/REF',
 'variants/SAS_AF',
 'variants/SVLEN',
 'variants/SVTYPE',
 'variants/TSD',
 'variants/altlen',
 'variants/is_snp',
 'variants/numalt']

##### Store dictionary array keys as constants

In [7]:
# Constants used for indexing VCF dictionary
SAMPLES = 'samples'
DATA = 'calldata/GT'
ID = 'variants/ID'
REF = 'variants/REF'
ALT = 'variants/ALT'
POS = 'variants/POS'
QUAL = 'variants/QUAL'
FILT = 'variants/FILTER_PASS'

# Constants to index for VCF information annotations


In [8]:
###What do data keys give:

#data_set['samples'] #samples from header lines   
## need to dtr max number of samples

#data_set['variants/CHROM']
# ## all data is for chromosome 9

#data_set['variants/POS']
## array([ 10163,  10273,  10327, ..., 102798, 102814, 102827], dtype=int32)  With gaps
##Consider gaps!!!!

#data_set['variants/QUAL']
## quality of data at each position

#data_set['calldata/GT']
##shows zygosity

#data_set[FILT]
## Gives true or false... 

#See GentotypeArray class documentation:
#https://scikit-allel.readthedocs.io/en/latest/model/ndarray.html#allel.model.ndarray.GenotypeArray

##### For Zygosity (Genotype) data

In [9]:
hetero_data = allel.GenotypeArray(data_set[DATA])
het_bool_array = hetero_data.is_het()

# Count heterozygous calls; Summing over variants (axis=0) or samples (axis=1)
hetero_data.count_het(axis=1)

array([10,  0,  1, ...,  0,  0,  1])

##### **For Frequency Filtering (code tut 2)**

In [45]:
# load all variant positions on chromosome 9
chrom = data_set['variants/CHROM']
pos_all = allel.SortedIndex(data_set['variants/POS'])
pos_all
##1748 positions in afr-small

0,1,2,3,4,...,1743,1744,1745,1746,1747
np.int32(10163),np.int32(10273),np.int32(10327),np.int32(10329),np.int32(10362),...,np.int32(102730),np.int32(102770),np.int32(102798),np.int32(102814),np.int32(102827)


In [55]:
vari_all = data_set['variants/AMR_AF']
vari_all


array([[nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       ...,
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan]], dtype=float32)

In [41]:
# Compress data??

genotype_all = allel.GenotypeChunkedArray(data_set['calldata/GT'])
genotype_all
# 503 indiv in the data

## Try to compress bigger dataset afr
#condition = genotype_all[:, :, 0]
#filtered_genotypes = genotype_all.compress(condition, axis=0)
#print(filtered_genotypes)

<GenotypeChunkedArray shape=(1748, 504, 2) dtype=int8
   nbytes=1.7M
   values=numpy.ndarray>

In [12]:
### load-in data from a specific chromosome

# Error: List, dictionary, set, etc. falls under the category of non hashable objects

# chrom = data_set['variants/CHROM']
# load all variant positions
# pos_all = allel.SortedIndex(data_set[chrom]['variants']['POS'])

In [28]:
#Metadat table of info???
#df_samples = pandas.read_csv('data/2015-09-21/samples.meta.txt', sep='\t', index_col='index')
#df_samples = allel.SortedIndex(

#these are empty arrays????
SNP_samples = data_set['variants/is_snp']
struct_info_1 = data_set['variants/SVLEN']
struct_info_2 = data_set['variants/SVTYPE']


array(['', '', '', ..., '', '', ''], dtype=object)

In [ ]:
def plot_fst(ac1, ac2, pos, blen=2000):
 
 fst, se, vb, _ = allel.stats.blockwise_hudson_fst(ac1, ac2, blen) #blen=blen
 
 # use the per-block average Fst as the Y coordinate
 y = vb
 
 # use the block centres as the X coordinate
 x = allel.stats.moving_statistic(pos, statistic=lambda v: (v[0] + v[-1]) / 2, size=blen)
 
 # plot
 fig, ax = plt.subplots(figsize=(10, 4))
 sns.despine(ax=ax, offset=5)
 ax.plot(x, y, 'k-', lw=.5)
 ax.set_ylabel('$F_{ST}$')
 ax.set_xlabel('Chromosome 9 position (bp)')
 ax.set_xlim(0, pos.max())

#plot_fst(ac1, ac2, pos)

### Joshu's Zygosity Class from dataWrapper.py

In [ ]:
# Conatins vcf query data and returns it in various formats
class VcfDataWrapper:
    def __init__(self, vcf_data:dict) -> None:

        # Tmp data size constriants
        # TODO REMOVE THESE
        max_vars = 5000
        max_samples = 300

        vcf_data[SAMPLES] = vcf_data[SAMPLES][:max_samples]
        vcf_data[DATA] = vcf_data[DATA][:max_vars,:max_samples,]
        vcf_data[REF] = vcf_data[REF][:max_vars]
        vcf_data[ALT] = vcf_data[ALT][:max_vars]
        vcf_data[ID] = vcf_data[ID][:max_vars]
        
        # ------------------------
        self.data = vcf_data
        self.gt_data = GTArr(self.data[DATA])
        self.n_samples = len(vcf_data[SAMPLES])
        self.n_variants = len(vcf_data[ID])

        # clear pre-constructed arrays
        self._zygos = None
        self._refs = None
        self._alts = None

    # Returns a matrix of zygosities for each sample variant.
    # 0 = no mutation, 1 = heterozygous, 2 = homozygous mutation, -1 = no-data
    def get_zygosity(self):
        if self._zygos is None: # zygosity not pre-computed
            gt_data = self.gt_data
            self._zygos = gt_data.is_hom_alt()*2 + gt_data.is_het()*1 + gt_data.is_missing()*(-1)
        return self._zygos.transpose()[::-1,:] # Flip order so that firs entry is on the top
    
    # Returns a list indicating the nucleotide type of ref sequences
    # 0 = multi-nucleotide, -1 is no-data
    def get_ref(self):
        if self._refs is None:
            self._refs = alleles_to_numbs(self.data[REF])
        return self._refs

    def get_alt(self):
        if self._alts is None:
            self._alts = np.array([alleles_to_numbs(alts) for alts in self.data[ALT]])
            filter_mask = np.array([np.max(self._alts,axis=0) >= 0][0]) # filter out empty columns
            self._alts = self._alts[:,filter_mask].transpose()[::-1,:] # Put samples on the rows in descending order
        return self._alts

# Converts and allele character/string to an intagetr
# 0 = multi-nucleotide, -1 = n0-data
def allele_to_numb(a:str):
    if len(a) > 1:
        return 0
    elif NUCLEOTIDE_DICT.keys().__contains__(a):
        return NUCLEOTIDE_DICT[a]
    else:
        return -1
# Converts an array of alleles to numbers
def alleles_to_numbs(alleles:np.array):
    return [allele_to_numb(a) for a in alleles]
